# COVID-Week-2-First-Submission

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Preparation
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Load the data
train=pd.read_csv('../input/covid19-global-forecasting-week-2/train.csv')
test=pd.read_csv('../input/covid19-global-forecasting-week-2/test.csv')
submission=pd.read_csv('../input/covid19-global-forecasting-week-2/submission.csv')

print(np.shape(train))
print(np.shape(test))

In [ ]:
train.head(10)

In [ ]:
train.describe()

In [ ]:
train.info() # Found that only Province_State has different row numbers

In [ ]:
train['Province_State'].value_counts()

In [ ]:
train['Province_State'].unique()[:50] # nan found

Use "No Data" to fill the ```nan```s in both train and test data sets.

In [ ]:
train['Province_State'].fillna('No Data',inplace=True)
test['Province_State'].fillna('No Data',inplace=True)

Transform to pandas datetime

In [ ]:
train['Date']= pd.to_datetime(train['Date']) 
test['Date']= pd.to_datetime(test['Date']) 

Individual second,minutes,hours,date help the algorithm predict better

In [ ]:
def create_time_features(df):
    df['date'] = df['Date']
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    return X

create_time_features(train)
create_time_features(test)

# drop the original date columns
train=train.drop(columns=['Date'],axis=1)
test=test.drop(columns=['Date'],axis=1)

train=train.drop(columns=['date'],axis=1)
test=test.drop(columns=['date'],axis=1)

In [ ]:
train.head()

In [ ]:
train.info()

There are 2 ```object``` columns left in the data: ```Province_State``` and ```Country_Region```. Convert them into dummy variables.

In [ ]:
train_dummies_Province_State = pd.get_dummies(train['Province_State'])
test_dummies_Province_State = pd.get_dummies(test['Province_State'])

train_dummies_Country_Region = pd.get_dummies(train['Country_Region'])
test_dummies_Country_Region = pd.get_dummies(test['Country_Region'])

train=train.drop(['Country_Region','Province_State','Id'],axis=1)
test=test.drop(['Country_Region','Province_State','ForecastId'],axis=1)

train=pd.concat([train,train_dummies_Province_State,train_dummies_Country_Region],axis=1)
test=pd.concat([test,test_dummies_Province_State,test_dummies_Country_Region],axis=1)

train.head()

Separate the features and targets.

In [ ]:
features=train.drop(['ConfirmedCases','Fatalities'],axis=1)
target1=train['ConfirmedCases']
target2=train['Fatalities']

print(features.shape)
print(target1.shape)
print(target2.shape)
print(test.shape)

Use Decison Tree as our Regression model

In [ ]:
model = DecisionTreeRegressor(criterion='mse', splitter='best')

For target1: ```ConfirmedCases```

In [ ]:
model.fit(features,target1)

In [ ]:
submission['ConfirmedCases'] = model.predict(test)

For target2: ```Fatalities```

In [ ]:
model.fit(features,target2)
submission['Fatalities'] = model.predict(test)

Round the results and then convert them to ```int```.

In [ ]:
submission.round().astype(int)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv',index=False)